# 1. Evolve only, steady state. 

In [1]:
%matplotlib inline 
import matplotlib
from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])
# Exact Solution
c = -16.0 #*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
iliescu = c*h*(0.5+sym.atan(g)/sym.pi)

ue = sym.simplify(iliescu)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = 0 #sym.diff(ue,t) Remove time dependence for now
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')


In [23]:
# %load main-EFR.py

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	errors = {'L2 norm': L2n, 'H1 norm': H1n}
	return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

def compute_extrema(u):
    print("max = " + str(np.amax(u.vector().get_local())))
    print("min = " + str(np.amin(u.vector().get_local())))

In [3]:
def evolve(nx, u_code, f_code, sigma, mu, velocity):
    degree = 2

    u_exact = Expression(u_code, degree = degree)
    
    f = Expression(f_code, degree = degree)

    mesh = UnitSquareMesh(nx,nx)

    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree)
    def boundary(x, on_boundary):
        return on_boundary
    
    bc = DirichletBC(Q, u_D, boundary)

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n = Function(Q)
    u_ = Function(Q)

    # Galerkin variational problem
    F = mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)
    bc.apply(A1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Step 1 Solve on Coarse Grid
    b = assemble(L)
    bc.apply(b)
    solve(A1, u_.vector(), b, 'gmres')
    
    errors = compute_errors(u_exact, u_, 0, mesh)
    print(errors)
    
    out_file_uexact = File("results/evolve_steady_u_exact_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_uexact << ue
    out_file_ubar = File("results/"+str(mu)+"evolve_steady_u_"+str(nx)+".pvd") 
    out_file_ubar << u_
    
    return u_

for nx in [25, 50, 100, 200, 400, 800]:
    print('nx = '+str(nx))
    evolve(nx, u_code, f_code, sigma, mu, b)

In [5]:
np.amin(u_25.vector().get_local())

-3.3370421946767843

In [6]:
np.amin(u_400.vector().get_local())

-0.99200903871018864

Help on GenericVector in module dolfin.cpp.la object:

class GenericVector(GenericTensor)
 |  Dummy docstring. Reason: Missing doxygen
 |  
 |  Method resolution order:
 |      GenericVector
 |      GenericTensor
 |      LinearAlgebraObject
 |      dolfin.cpp.common.Variable
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      x.__add__(y) <==> x+y
 |  
 |  __contains__(self, value)
 |  
 |  __delitem__(self, i)
 |  
 |  __delslice__(self, i, j)
 |  
 |  __div__ = __truediv__(self, other)
 |  
 |  __eq__(self, value)
 |  
 |  __ge__(self, value)
 |  
 |  __getitem__(self, indices)
 |      Return values corresponding to the given local indices
 |  
 |  __getslice__(self, i, j)
 |  
 |  __gt__(self, value)
 |  
 |  __iadd__(self, other)
 |      x.__iadd__(y) <==> x+y
 |  
 |  __idiv__ = __itruediv__(self, other)
 |  
 |  __imul__(self, other)
 |      x.__imul__(y) <==> x*y
 |  
 |  __init__(self, *args, **kwargs)
 |  
 |  __isub__(self, other)
 

In [8]:
u_exact = evolve(25, u_code, f_code, sigma, mu, b)

L2, 0.575963540234, H1, 77.7640877026, t, 0



In [9]:
np.amax(u_exact.vector().get_local())

3.0058415614709171

In [15]:
list2 = []
for ue in [u_25, u_400]:
    list2.append(np.amax(ue.vector().get_local()))
    list2.append(np.amin(ue.vector().get_local()))

In [16]:
list2

[3.0058415614709171,
 -3.3370421946767843,
 0.00095250536929038666,
 -0.99200903871018864]

max = 0.00095250536929
min = -0.99200903871
